## 1. KBO

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# input : url
# output : players data

def extract_datas_hitters(url):
    response = requests.get(url)

    colnames = []
    rows = []

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        # column name
        header = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > thead')[0]
        colnames = header.text.splitlines()[2:-1]

        # Row : each player
        elements = soup.find('tbody').find_all('tr')
        for player in elements:
            rows.append(player.text.splitlines()[1:]) #index 제외
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

    df = pd.DataFrame(1, columns=colnames)

    return(df)



In [ ]:
# example URL
url = "https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx"

hit_2023_example = extract_datas_hitters(url)
hit_2023_example

,순위,선수명,팀명,AVG,G,PA,AB,R,H,2B,3B,HR,TB,RBI,SAC,SF
0,1,손아섭,NC,0.339,140,609,551,97,187,36,3,5,244,65,1,5
1,2,구자욱,삼성,0.336,119,515,453,65,152,37,1,11,224,71,2,3
2,3,김혜성,키움,0.335,137,621,556,104,186,29,6,7,248,57,0,5
3,4,홍창기,LG,0.332,141,643,524,109,174,35,2,1,216,65,3,6
4,5,에레디아,SSG,0.323,122,523,473,76,153,29,0,12,218,76,1,1
5,6,김선빈,KIA,0.320,119,473,419,41,134,16,0,0,150,48,8,3
6,7,박건우,NC,0.319,130,533,458,70,146,34,2,12,220,85,1,9
7,8,박민우,NC,0.316,124,509,452,76,143,20,7,2,183,46,5,3
8,9,오스틴,LG,0.313,139,583,520,87,163,29,4,23,269,95,0,7
9,10,전준우,롯데,0.312,138,559,493,80,154,21,3,17,232,77,0,7


---

### PlayerId 존재하는 것만 뽑아보기

In [ ]:
# 선수 id 예시로 뽑아보기

base_url = 'https://www.koreabaseball.com/Record/Player/PitcherDetail/Basic.aspx?playerId='

id_range = list(range(40000, 40100, 1))

id_list = []

for i in id_range:

    url = base_url + str(i)
    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        # column name
        elements = soup.select('#cphContents_cphContents_cphContents_playerProfile_lblName')

        if len(elements[0].text) != 0:
            id_list.append(i)

    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


id_list

[40001, 40002, 40003, 40005, 40006, 40007]

In [4]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

In [5]:
base_url = 'https://www.koreabaseball.com/Record/Player/PitcherDetail/Total.aspx?playerId=' #선수 조회 통산기록 페이지 (일단 투수 기준으로)
id_range = list(range(1, 100000, 1))

def check_player_id(player_id):
    url = base_url + str(player_id)
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        elements = soup.select('#cphContents_cphContents_cphContents_playerProfile_lblName')
        if elements and len(elements[0].text) != 0:
            name = elements[0].text
            return name, player_id
    except requests.RequestException as e:
        print(f"Error for ID {player_id}: {e}")
    return None

valid_player = []
max_workers = os.cpu_count() #8

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(check_player_id, player_id) for player_id in id_range]
    for i, future in enumerate(as_completed(futures)):
        result = future.result()
        if result:
            row = [result[0], result[1]]
            valid_player.append(row)

        if i % 500 == 0:
            print(f"Processed {i}th requests.")

Processed 0th requests.


In [ ]:
#혹시 100000 이후에 있는지 확인

base_url = 'https://www.koreabaseball.com/Record/Player/PitcherDetail/Total.aspx?playerId=' #선수 조회 통산기록 페이지 (일단 투수 기준으로 하는데 어차피 전부 다 나옴)
id_range = list(range(100000, 110000, 1))

def check_player_id(player_id):
    url = base_url + str(player_id)
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        elements = soup.select('#cphContents_cphContents_cphContents_playerProfile_lblName')
        if elements and len(elements[0].text) != 0:
            name = elements[0].text
            return name, player_id
    except requests.RequestException as e:
        print(f"Error for ID {player_id}: {e}")
    return None

valid_player_over = []
max_workers = os.cpu_count() #8

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(check_player_id, player_id) for player_id in id_range]
    for i, future in enumerate(as_completed(futures)):
        result = future.result()
        if result:
            row = [result[0], result[1]]
            valid_player_over.append(row)

        if i % 500 == 0:
            print(f"Processed {i}th requests.")

valid_player_over

Processed 0th requests.
Processed 500th requests.
Processed 1000th requests.
Processed 1500th requests.
Processed 2000th requests.
Processed 2500th requests.
Processed 3000th requests.
Processed 3500th requests.
Processed 4000th requests.
Processed 4500th requests.
Processed 5000th requests.
Processed 5500th requests.
Processed 6000th requests.
Processed 6500th requests.
Processed 7000th requests.
Processed 7500th requests.
Processed 8000th requests.
Processed 8500th requests.
Processed 9000th requests.
Processed 9500th requests.


[]

In [ ]:
valid_player_df = pd.DataFrame(valid_player, columns=['name', 'playerid'])
valid_player_df.to_csv('./validPlayerId.csv', index=False)
valid_player_df

,name,playerid
0,차영화,10005
1,황태환,10082
2,황기선,10182
3,조창수,20002
4,김광현,20003
...,...,...
5734,김병준,99742
5735,방재훈,99745
5736,전하진,99767
5737,이진영,99810


---

### 타자 정보 뽑아내기

In [8]:
valid_player = pd.read_csv('/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/validPlayerId.csv')
id_to_name = valid_player.set_index('playerid')['name'].to_dict()

In [14]:
base_url = 'https://www.koreabaseball.com/Record/Player/HitterDetail/Total.aspx?playerId=' #선수 조회 통산기록 페이지
id_range = valid_player['playerid'] #for test : [62404, 77532]


hitter_head = ['이름', 'id', '연도', '팀명', '타율', '경기', '타석', '타수', '득점',
               '안타', '2루타', '3루타', '홈런', '루타', '타점',
               '도루', '도루실패', '볼넷', '사구', '삼진', '병살타', '장타율', '출루율', '실책', '생년월일']

def extract_hitter(player_id):
    url = base_url + str(player_id)
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        elements = soup.find('tbody').find_all('tr')
        if len(elements) != 0:
            rows = []
            for element in elements:
                row = [id_to_name[player_id], player_id]
                row.extend(element.text.splitlines()[1:])
                birth = soup.select_one('#cphContents_cphContents_cphContents_playerProfile_lblBirthday')
                birth = birth.text
                row = row + [birth]
                #print(row)
                rows.append(row)
                
            return rows
    except requests.RequestException as e:
        print(f"Error for ID {player_id}: {e}")
    return None

max_workers = os.cpu_count() #8
hitterRecords = []


with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(extract_hitter, player_id) for player_id in id_range]
    for i, future in enumerate(as_completed(futures)):
        result = future.result()
        if result:
            hitterRecords.append(result)
        if i % 500 == 0:
            print(f"Processed {i}th requests.")

hitterRecords = [record for player in hitterRecords for record in player]
#len(hitterRecords[0]), len(hitter_head)
hitter_df = pd.DataFrame(hitterRecords, columns=hitter_head)

['김웅빈', 65898, '2016', '넥센', '0.429', '10', '15', '14', '4', '6', '0', '0', '1', '9', '4', '0', '0', '0', '0', '3', '0', '0.643', '0.400', '2', '1996년 02월 09일']['최승민', 65905, '2019', 'NC', '0.444', '24', '10', '9', '5', '4', '1', '0', '0', '5', '1', '6', '2', '1', '0', '2', '0', '0.556', '0.500', '0', '1996년 07월 01일']
['최승민', 65905, '2021', 'NC', '0.226', '48', '34', '31', '8', '7', '1', '0', '0', '8', '1', '4', '3', '1', '0', '7', '0', '0.258', '0.250', '0', '1996년 07월 01일']
['최승민', 65905, '2022', 'NC', '0.324', '44', '34', '34', '19', '11', '0', '0', '0', '11', '3', '6', '3', '0', '0', '7', '1', '0.324', '0.324', '1', '1996년 07월 01일']
['최승민', 65905, '2023', 'LG', '0.071', '38', '15', '14', '10', '1', '0', '0', '0', '1', '1', '8', '3', '1', '0', '4', '1', '0.071', '0.133', '0', '1996년 07월 01일']
['류진욱', 65949, '2021', 'NC', '-', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '-', '-', '0', '1996년 10월 10일']
['류진욱', 65949, '2022', 'NC', '-', '1', '0', '0'

In [19]:
from datetime import datetime

hitter_df['생년월일'] = hitter_df['생년월일'].apply(lambda x: datetime.strptime(x, '%Y년 %m월 %d일').strftime('%Y-%m-%d'))
hitter_df.to_csv('hitterRecords.csv', index=False)
hitter_df

,이름,id,연도,팀명,타율,경기,타석,타수,득점,안타,...,도루,도루실패,볼넷,사구,삼진,병살타,장타율,출루율,실책,생년월일
0,차영화,10005,1982,해태,0.259,76,323,282,46,73,...,32,10,31,3,20,3,0.000,0.000,5,1957-06-27
1,차영화,10005,1983,해태,0.266,86,252,229,20,61,...,16,11,13,2,17,3,0.000,0.000,7,1957-06-27
2,차영화,10005,1984,해태,0.204,70,185,162,16,33,...,11,6,15,1,24,2,0.000,0.000,11,1957-06-27
3,차영화,10005,1985,해태,0.239,107,266,238,21,57,...,13,5,11,1,21,6,0.000,0.000,7,1957-06-27
4,차영화,10005,1986,해태,0.199,85,192,176,18,35,...,3,5,9,1,9,5,0.000,0.000,8,1957-06-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10843,이진영,99810,2014,LG,0.325,119,468,412,53,134,...,4,3,45,0,48,10,0.425,0.384,0,1980-06-15
10844,이진영,99810,2015,LG,0.256,103,350,301,32,77,...,2,4,42,3,55,10,0.369,0.351,1,1980-06-15
10845,이진영,99810,2016,KT,0.332,115,424,371,49,123,...,2,5,45,3,44,13,0.480,0.403,0,1980-06-15
10846,이진영,99810,2017,KT,0.289,103,285,263,28,76,...,0,0,17,1,34,3,0.418,0.335,0,1980-06-15


---

### 투수 정보 뽑아내기

In [26]:
base_url = 'https://www.koreabaseball.com/Record/Player/PitcherDetail/Total.aspx?playerId=' #선수 조회 통산기록 페이지 (일단 투수 기준으로)
#id_range = [65933, 68701]
id_range = valid_player['playerid']

pitcher_head = ['이름', 'id', '연도', '팀명', '평균자책점', '경기', '완투', '완봉', '승리', '패배', '세이브', '홀드', '승률',
               '타자수', '이닝', '피안타', '피홈런', '볼넷', '사구', '삼진', '실점', '자책점', '생년월일']

def extract_pitcher(player_id):
    url = base_url + str(player_id)
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        elements = soup.find('tbody').find_all('tr')
        if len(elements) != 0:
            rows = []
            for element in elements:
                row = [id_to_name[player_id], player_id]
                row.extend(element.text.splitlines()[1:])
                birth = soup.select_one('#cphContents_cphContents_cphContents_playerProfile_lblBirthday')
                birth = birth.text
                row = row + [birth]
                rows.append(row)
            return rows
    except requests.RequestException as e:
        print(f"Error for ID {player_id}: {e}")
    return None

max_workers = os.cpu_count() #8
pitcherRecords = []


with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(extract_pitcher, player_id) for player_id in id_range]
    for i, future in enumerate(as_completed(futures)):
        result = future.result()
        if result:
            pitcherRecords.append(result)
        if i % 500 == 0:
            print(f"Processed {i}th requests.")

pitcherRecords = [record for player in pitcherRecords for record in player]
pitcher_df = pd.DataFrame(pitcherRecords, columns=pitcher_head)

Processed 0th requests.
Processed 500th requests.
Processed 1000th requests.
Processed 1500th requests.
Processed 2000th requests.
Processed 2500th requests.
Processed 3000th requests.
Processed 3500th requests.
Processed 4000th requests.
Processed 4500th requests.
Processed 5000th requests.
Processed 5500th requests.


In [27]:
pitcher_df

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,타자수,이닝,피안타,피홈런,볼넷,사구,삼진,실점,자책점,생년월일
0,황기선,10182,1983,해태,2.31,19,0,0,4,4,...,265,66 1/3,50,3,14,7,35,27,17,1958년 06월 02일
1,황기선,10182,1984,해태,3.76,38,2,0,4,7,...,536,124 1/3,110,10,55,9,65,70,52,1958년 06월 02일
2,황기선,10182,1985,해태,6.49,17,0,0,0,1,...,159,34 2/3,40,3,19,2,15,27,25,1958년 06월 02일
3,황기선,10182,1986,OB,4.44,11,0,0,0,0,...,108,24 1/3,26,1,15,2,6,14,12,1958년 06월 02일
4,황기선,10182,1987,OB,0.00,2,0,0,0,0,...,9,1 2/3,3,0,0,0,2,3,0,1958년 06월 02일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,박정진,99737,2015,한화,3.09,76,0,0,6,1,...,411,96,84,8,39,5,92,37,33,1976년 05월 27일
7099,박정진,99737,2016,한화,5.57,77,0,0,4,4,...,374,84,96,11,33,2,85,58,52,1976년 05월 27일
7100,박정진,99737,2017,한화,3.94,55,0,0,3,2,...,205,48,38,6,24,3,51,28,21,1976년 05월 27일
7101,전하진,99767,2000,한화,6.17,7,0,0,0,0,...,54,11 2/3,14,2,5,1,8,8,8,1979년 02월 11일


In [28]:
pitcher_df['생년월일'] = pitcher_df['생년월일'].apply(lambda x: datetime.strptime(x, '%Y년 %m월 %d일').strftime('%Y-%m-%d'))

pitcher_df.to_csv('/Users/a05/Library/CloudStorage/GoogleDrive-hithisis05@gmail.com/내 드라이브/EDA_24_1/datasets/pitcherRecords.csv', index=False)
pitcher_df.to_csv('pitcherRecords.csv', index=False)
pitcher_df

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,타자수,이닝,피안타,피홈런,볼넷,사구,삼진,실점,자책점,생년월일
0,황기선,10182,1983,해태,2.31,19,0,0,4,4,...,265,66 1/3,50,3,14,7,35,27,17,1958-06-02
1,황기선,10182,1984,해태,3.76,38,2,0,4,7,...,536,124 1/3,110,10,55,9,65,70,52,1958-06-02
2,황기선,10182,1985,해태,6.49,17,0,0,0,1,...,159,34 2/3,40,3,19,2,15,27,25,1958-06-02
3,황기선,10182,1986,OB,4.44,11,0,0,0,0,...,108,24 1/3,26,1,15,2,6,14,12,1958-06-02
4,황기선,10182,1987,OB,0.00,2,0,0,0,0,...,9,1 2/3,3,0,0,0,2,3,0,1958-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7098,박정진,99737,2015,한화,3.09,76,0,0,6,1,...,411,96,84,8,39,5,92,37,33,1976-05-27
7099,박정진,99737,2016,한화,5.57,77,0,0,4,4,...,374,84,96,11,33,2,85,58,52,1976-05-27
7100,박정진,99737,2017,한화,3.94,55,0,0,3,2,...,205,48,38,6,24,3,51,28,21,1976-05-27
7101,전하진,99767,2000,한화,6.17,7,0,0,0,0,...,54,11 2/3,14,2,5,1,8,8,8,1979-02-11


In [ ]:
pitcher_df[pitcher_df['이름'] == '류현진']

,이름,id,연도,팀명,평균자책점,경기,완투,완봉,승리,패배,...,승률,타자수,이닝,피안타,피홈런,볼넷,사구,삼진,실점,자책점
3671,류현진,76715,2006,한화,2.23,30,6,1,18,6,...,0.750,800,201 2/3,159,11,52,2,204,57,50
3672,류현진,76715,2007,한화,2.94,30,6,1,17,7,...,0.708,867,211,195,15,68,3,178,74,69
3673,류현진,76715,2008,한화,3.31,26,2,1,14,7,...,0.667,684,165 2/3,144,12,67,1,143,66,61
3674,류현진,76715,2009,한화,3.57,28,4,2,13,12,...,0.520,793,189 1/3,180,19,67,2,188,80,75
3675,류현진,76715,2010,한화,1.82,25,5,3,16,4,...,0.800,756,192 2/3,149,11,45,9,187,42,39
3676,류현진,76715,2011,한화,3.36,24,3,0,11,7,...,0.611,520,126,101,12,38,1,128,54,47
3677,류현진,76715,2012,한화,2.66,27,1,0,9,9,...,0.500,734,182 2/3,153,12,46,5,210,58,54


In [1]:
import pandas as pd

In [ ]:
base_url = 'http://www.statiz.co.kr/team.php?opt=0&sopt=3&year=2023&team='

In [17]:
base_url = 'http://www.statiz.co.kr/team.php?opt=0&sopt=3&year=2023&team='
team_list = ['한화', '두산', 'SSG', 'KT', '삼성', 'LG', '키움', '롯데', 'NC', 'KIA']

position = []
for team in team_list:

    url = base_url + str(team)
    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        leftout = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(1)')
        middleout = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(2)')
        rightout = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(3)')
        
        leftin = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(4)')
        leftmiddlein = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(5)')
        rightmiddlein = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(6)')
        rightin = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(7)')
        
        pitchall = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(8)')
        pitchpart = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(11)')
        
        catcher = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(9)')
        dhit = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(2) > div > div:nth-child(2) > div > div.box-body.no-padding > div > div:nth-child(10)')
        
        poslit = [leftout, middleout, rightout, leftin, leftmiddlein, rightmiddlein, rightin, pitchall, pitchpart, catcher, dhit]
        posenc = ['좌익수', '중견수', '우익수', '3루수', '유격수', '2루수', '1루수', '선발투수', '중계투수', '포수', '지명타자']
        for posname, pos in enumerate(poslit):
            for element in pos:
                links = element.find_all('a')
                for link in links:
                    item = [team, link.text, posenc[posname]]
                    position.append(item)

    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


position

[['한화', '최인호', '좌익수'],
 ['한화', '노수광', '좌익수'],
 ['한화', '윌리엄스', '좌익수'],
 ['한화', '문현빈', '중견수'],
 ['한화', '이원석', '중견수'],
 ['한화', '이상혁', '중견수'],
 ['한화', '이진영', '우익수'],
 ['한화', '김태연', '우익수'],
 ['한화', '장운호', '우익수'],
 ['한화', '노시환', '3루수'],
 ['한화', '이도윤', '유격수'],
 ['한화', '오선진', '유격수'],
 ['한화', '이민준', '유격수'],
 ['한화', '정은원', '2루수'],
 ['한화', '채은성', '1루수'],
 ['한화', '김인환', '1루수'],
 ['한화', '페냐', '선발투수'],
 ['한화', '문동주', '선발투수'],
 ['한화', '산체스', '선발투수'],
 ['한화', '김기중', '선발투수'],
 ['한화', '송윤준', '선발투수'],
 ['한화', '주현상', '중계투수'],
 ['한화', '이태양', '중계투수'],
 ['한화', '윤대경', '중계투수'],
 ['한화', '한승주', '중계투수'],
 ['한화', '장시환', '중계투수'],
 ['한화', '최재훈', '포수'],
 ['한화', '이재용', '포수'],
 ['한화', '박상언', '포수'],
 ['한화', '유상빈', '지명타자'],
 ['한화', '이성곤', '지명타자'],
 ['한화', '김건', '지명타자'],
 ['두산', '로하스', '좌익수'],
 ['두산', '송승환', '좌익수'],
 ['두산', '정수빈', '중견수'],
 ['두산', '김인태', '우익수'],
 ['두산', '양찬열', '우익수'],
 ['두산', '조수행', '우익수'],
 ['두산', '허경민', '3루수'],
 ['두산', '박준영', '3루수'],
 ['두산', '전민재', '3루수'],
 ['두산', '김재호', '유격수'],
 ['두산', '권민석', '유격수'],
 [

In [18]:
playerPositions_2023 = pd.DataFrame(position, columns= ['팀명', '이름', '포지션'])

In [20]:
playerPositions_2023.to_csv('playerPositions_2023.csv')

---

### 경기별 데이터 뽑아보기

#### 경기 기록 뽑기

In [3]:
%pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [4]:
%pip install webdriver_manager

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

In [101]:
#MAC 전용코드

driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))

base_url = 'https://m.sports.naver.com/kbaseball/schedule/index?category=kbo&date='
monthstarts = ['2023-03-13', '2023-04-01', '2023-05-02', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01', '2023-11-02']

match_2023 = []
for month in monthstarts:
    url = base_url + month
    driver.get(url)

    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    selector = "#content > div > div:nth-child(5) > div"
    parent = soup.select_one(selector)
    children = parent.find_all('div', class_ ='ScheduleLeagueType_match_list_group__18ML9')

    data_match = []

    for child in children:
        date = child.find('em', class_ = 'ScheduleLeagueType_title__2Kalm').text
        dformat = re.search(r"(\d+)월 (\d+)일", date)
        month = dformat.group(1)
        day = dformat.group(2)

        matchdata = []
        matches = child.find_all('li', class_ = 'MatchBox_match_item__3_D0Q MatchBox_type_baseball__3VkMB type_end')

        for match in matches:
            teams = match.find_all('strong', class_ ='MatchBoxTeamArea_team__3aB4O')
            #MatchBoxTeamArea_team_name__2G9t1
            scores = match.find_all('strong', class_='MatchBoxTeamArea_score__1_YFB') #취소 경기는 자동 제외
            teams = [team.text.strip() for team in teams]
            scores = [score.text.strip() for score in scores]
            matchdata.append([teams, scores])

        for teams, scores in matchdata:
            finalmatchdata= [month, day, *teams, *scores]
            data_match.append(finalmatchdata)
    match_2023 = match_2023 + data_match

driver.quit()

In [3]:
def year_matchdata(driver, monthstarts, POmonth, POday):

    # -------------------
    # example : 2023
    # monthstarts = ['2023-04-01', '2023-05-02', '2023-06-01',
    #                '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01']
    # POmonth, POday  : 플레이오프 시작날짜 (ex: 1019) -> 이후 데이터는 제외
    # -------------------

    base_url = 'https://m.sports.naver.com/kbaseball/schedule/index?category=kbo&date='
    match_year = []
    for month in monthstarts:
        url = base_url + month
        driver.get(url)

        time.sleep(1)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        selector = "#content > div > div:nth-child(5) > div"
        parent = soup.select_one(selector)
        children = parent.find_all('div', class_ ='ScheduleLeagueType_match_list_group__18ML9')

        data_match = []

        for child in children:
            date = child.find('em', class_ = 'ScheduleLeagueType_title__2Kalm').text
            dformat = re.search(r"(\d+)월 (\d+)일", date)
            month = dformat.group(1)
            day = dformat.group(2)

            matchdata = []
            matches = child.find_all('li', class_ = 'MatchBox_match_item__3_D0Q MatchBox_type_baseball__3VkMB type_end')

            for match in matches:
                teams = match.find_all('strong', class_ ='MatchBoxTeamArea_team__3aB4O')
                scores = match.find_all('strong', class_='MatchBoxTeamArea_score__1_YFB') #취소 경기는 자동 제외
                teams = [team.text.strip() for team in teams]
                scores = [score.text.strip() for score in scores]
                matchdata.append([teams, scores])

            for teams, scores in matchdata:
                finalmatchdata= [month, day, *teams, *scores]
                data_match.append(finalmatchdata)
        match_year = match_year + data_match

    driver.quit()

    result = pd.DataFrame(match_year, columns=['월', '일', '원정', '홈', '원정점수', '홈점수'])

    # 정규시즌 이외 데이터는 제외
    result = result.loc[(result['월'].astype(int) < POmonth) | ((result['월'].astype(int) == POmonth) & (result['일'].astype(int) < POday))]

    return result


In [123]:
month2023 = ['2023-04-01', '2023-05-02', '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01', '2023-10-01']
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
match_2023 = year_matchdata(driver, month2023, 10, 19)

match_2023

,월,일,원정,홈,원정점수,홈점수
0,4,1,한화,키움,2,3
1,4,1,KIA,SSG,1,4
2,4,1,LG,KT,6,11
3,4,1,롯데,두산,10,12
4,4,1,NC,삼성,8,0
...,...,...,...,...,...,...
716,10,16,롯데,한화,7,2
717,10,16,NC,KIA,2,4
718,10,16,SSG,두산,3,2
719,10,17,NC,KIA,1,7


#### 당해 경기 데이터 변환

In [126]:
## 경기수 잘 뽑았는지 확인

validteam = ['KIA', 'SSG', '키움', '한화', 'LG', 'KT', 'NC', '삼성', '롯데', '두산']
for team in validteam:
    print(team, ":",len(match_2023.loc[match_2023['원정'] == team]), len(match_2023.loc[match_2023['홈'] == team]))
    

KIA : 73 71
SSG : 71 73
키움 : 73 71
한화 : 71 73
LG : 73 71
KT : 71 73
NC : 73 71
삼성 : 73 71
롯데 : 71 73
두산 : 71 73


In [245]:
## 잘 뽑혔을 경우 저장

match_2023.to_csv('match_2023.csv')

In [5]:
match_2023 = pd.read_csv('match_2023.csv')

In [4]:
from collections import Counter

def match_to_id(year, table):
    # --------------
    # input : year_matchdata return value
    # output : match id list ex)20230313HTHH02023
    # --------------

    # 연도별로 팀 이름이 다르기 때문에
    teamid = {'KIA' : 'HT', 'SSG' : 'SK', '키움' : 'WO', '한화' : 'HH', 'LG' : 'LG',
              'KT' : 'KT', 'NC' : 'NC', '삼성': 'SS', '롯데' :'LT', '두산' : 'OB', 'SK' : 'SK', '넥센' : 'WO', 'kt' : 'KT'}

    # 여기 없는 row들은 제외하기
    table = table.loc[table['원정'].isin(teamid.keys())]

    result_list = []
    for _, row in table.iterrows():
        away_team_id = teamid[row['원정']]
        home_team_id = teamid[row['홈']]
        date = f"{str(row['월']).zfill(2)}{str(row['일']).zfill(2)}"
        new_format = f"{year}{date}{away_team_id}{home_team_id}" + str(0) + str(year)
        result_list.append(new_format)
    

    element_counts = Counter(result_list)
    duplicates1 = [item for item, count in element_counts.items() if count == 2]
    print("중복 값 1:", duplicates1)
    
    duplicates2 = [item for item, count in element_counts.items() if count == 3]
    print("중복 값 2:", duplicates2)
    
    
    
    #updated = [s[:-5] + '1' + s[-4:] for s in duplicates1]

    # match_2023_error와 match_2023_error_updated를 딕셔너리로 매핑
    #error_mapping = dict(zip(duplicates, updated))

    # matchid_2023에서 해당하는 값을 변경
    result_list = set(result_list)
    #result_list_updated = sorted([error_mapping.get(item, item) for item in result_list])
    
    return(result_list, duplicates1, duplicates2)

In [216]:
# 중복된 행 찾기
duplicates = match_2023[match_2023.duplicated(subset=['월', '일', '원정', '홈'], keep=False)]

# 중복된 행 출력
print(duplicates)

     Unnamed: 0   월   일   원정    홈  원정점수  홈점수
584         584   9   9   한화   키움    11    2
585         585   9   9   LG  KIA     6    7
586         586   9   9   롯데   NC     5    2
587         587   9   9   삼성   두산     5    1
589         589   9   9   삼성   두산     0    4
590         590   9   9   롯데   NC     5    6
591         591   9   9   한화   키움     3    1
592         592   9   9   LG  KIA     7   12
611         611   9  17   KT   한화     7    0
614         614   9  17  SSG   LG     3    8
615         615   9  17   KT   한화     3    1
616         616   9  17  SSG   LG     5    9
643         643   9  27  KIA   NC     0    7
644         644   9  27   KT   LG     0    4
645         645   9  27   두산  SSG     7    7
646         646   9  27   삼성   한화    11    3
647         647   9  27  KIA   NC     6    1
648         648   9  27   KT   LG     0    3
649         649   9  27   삼성   한화     0    4
650         650   9  27   두산  SSG     3    6
668         668  10   2   삼성   롯데     9    8
670       

In [230]:
matchid_2023, dup_2023_1, dup_2023_2 = match_to_id(2023, match_2023)
len(matchid_2023)
dup_2023_1

중복 값 1: ['20230909HHWO02023', '20230909LGHT02023', '20230909LTNC02023', '20230909SSOB02023', '20230917KTHH02023', '20230917SKLG02023', '20230927HTNC02023', '20230927KTLG02023', '20230927OBSK02023', '20230927SSHH02023', '20231002SSLT02023', '20231004HTKT02023']
중복 값 2: []


['20230909HHWO02023',
 '20230909LGHT02023',
 '20230909LTNC02023',
 '20230909SSOB02023',
 '20230917KTHH02023',
 '20230917SKLG02023',
 '20230927HTNC02023',
 '20230927KTLG02023',
 '20230927OBSK02023',
 '20230927SSHH02023',
 '20231002SSLT02023',
 '20231004HTKT02023']

In [231]:
dup1 = [s[:-5] + '1' + s[-4:] for s in dup_2023_1]
dup2 = [s[:-5] + '2' + s[-4:] for s in dup_2023_1]

error_mapping = dict(zip(dup_2023_1, dup1))
matchid_2023 = sorted([error_mapping.get(item, item) for item in matchid_2023])
matchid_2023 = matchid_2023 + dup2
len(matchid_2023)

720


#### 경기 별 투타 데이터 뽑기

In [5]:
# 전처리 함수

def hitterPreprocess(t, matchid):

    table = t.copy()
    table['타자명'] = table['타자명'].str.replace('·', '')
    table['타자명'] = table['타자명'].str.replace(r'(지명타자|2루수|중견수|유격수|좌익수|우익수|3루수|포수|1루수|대타|투수)', '', regex=True)
    table['타자명'] = table['타자명'].str.replace(r'(지명|2루|중견|유격|좌익|우익|3루|포|1루|대타|투)', '', regex=True)
    table['타자명'] = table['타자명'].str.replace(r'\d+번 ', '', regex=True)
    table['타자명'] = table['타자명'].str.replace(r'(타자|교체)', '', regex=True)
    table.columns = [col.replace('내림차순 정렬', '').strip() for col in table.columns]
    table = table.loc[:, :'타율']
    table['타율'] = table['타율'].apply(lambda x: f"{x:.3f}")
    table = table[:-1]
    table['연도'] = matchid[0:4]
    table['월'] = matchid[4:6]
    table['일'] = matchid[6:8]
    
    return table

def pitcherPreprocess(t, matchid):
   
    table = t.copy()
    table.columns = [col.replace('내림차순 정렬', '').strip() for col in table.columns]
    table['투수명'] = table['투수명'].str.replace(r'(홀|승|패|세)$', '', regex=True)
    table = table[:-1]
    table['연도'] = matchid[0:4]
    table['월'] = matchid[4:6]
    table['일'] = matchid[6:8]
    
    return table

In [6]:
def process_player_data(soup, matchid, team_type):
    try:
        hitterhtml = soup.find_all('table', class_='PlayerRecord_record_table__19F6_ PlayerRecord_type_batter__1rHnm eg-flick-panel')
        pitcherhtml = soup.find_all('table', class_='PlayerRecord_record_table__19F6_ PlayerRecord_type_pitcher__1TYaz eg-flick-panel')

        hitter = pd.read_html(str(hitterhtml[0]))[0]
        pitcher = pd.read_html(str(pitcherhtml[0]))[0]

        hitters = hitterPreprocess(hitter, matchid)
        pitchers = pitcherPreprocess(pitcher, matchid)

        hitters['소속팀'] = soup.find('div', 'ScoreBox_away__26sht').text
        hitters['상대팀'] = soup.find('div', class_='ScoreBox_home__2uCuR').text
        hitters['경기장'] = team_type
        hitters['matchid'] = matchid

        pitchers['소속팀'] = soup.find('div', 'ScoreBox_away__26sht').text
        pitchers['상대팀'] = soup.find('div', class_='ScoreBox_home__2uCuR').text
        pitchers['경기장'] = team_type
        pitchers['matchid'] = matchid
        
        return hitters, pitchers

    except IndexError:
        return pd.DataFrame(), pd.DataFrame()

In [7]:
from selenium.common.exceptions import NoSuchElementException

def extractPlayerRecords(driver, matchid):

    url = 'https://m.sports.naver.com/game/' + matchid + '/record'
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        driver.get(url)

        time.sleep(0.8)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 홈팀
        hitters, pitchers = process_player_data(soup, matchid, '홈')
            
        button = driver.find_element(By.CLASS_NAME, "PlayerRecord_tab__3DKLH")
        button.click()
        time.sleep(0.8)
        
        html2 = driver.page_source
        soup2 = BeautifulSoup(html2, 'html.parser')
        
        # 원정팀
        hitters2, pitchers2 = process_player_data(soup2, matchid, '원정')
            
        h_total =  pd.concat([hitters, hitters2], ignore_index=True)
        p_total = pd.concat([pitchers, pitchers2], ignore_index=True)
        
    except NoSuchElementException:
            print(f"Element not found for matchid {matchid}")
            h_total = pd.DataFrame()
            p_total = pd.DataFrame()
            
    return h_total, p_total

In [226]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
hitterrecords =  pd.DataFrame()
pitcherrecords = pd.DataFrame()
for match in ['20230701HHSS02023', '20230418SSWO02023']:
    h, p = extractPlayerRecords(driver, match)
    hitterrecords = pd.concat([hitterrecords, h], ignore_index=True)
    pitcherrecords = pd.concat([pitcherrecords, p], ignore_index=True)

driver.quit()

In [9]:
# 전체 경기수 / 6

def process_match(matchid):
    hitterrecords =  pd.DataFrame()
    pitcherrecords = pd.DataFrame()
    
    driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
    for match in matchid:
        
        h, p = extractPlayerRecords(driver, match)
        hitterrecords = pd.concat([hitterrecords, h], ignore_index=True)
        pitcherrecords = pd.concat([pitcherrecords, p], ignore_index=True)
    
    driver.quit()
        
    return hitterrecords, pitcherrecords

In [10]:
## 최종 함수
from concurrent.futures import ThreadPoolExecutor, as_completed


def MatchPlayerRecords(matchidlist, batchsize = 120):
    max_workers = 6  # 사용 가능한 최대 CPU 코어 수
    hitter_records = []
    pitcher_records = []

    batches = [matchidlist[i:i + batchsize] for i in range(0, len(matchidlist), batchsize)]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # 각 matchid에 대해 extractPlayerRecords 함수를 병렬로 실행
        future_to_batch = {executor.submit(process_match, batch): batch for batch in batches}
        
        for i, future in enumerate(as_completed(future_to_batch), 1):
            result = future.result()
            if result:
                h, p = result
                hitter_records.append(h)
                pitcher_records.append(p)
            
            if i % (len(batches) // max_workers) == 0: 
                print(f"Completed batch {i} of {len(batches)}")

    # 결과를 병합
    hitterrecords = pd.concat(hitter_records, ignore_index=True)
    pitcherrecords = pd.concat(pitcher_records, ignore_index=True)

    return hitterrecords, pitcherrecords

#### 2023년 최종데이터

In [232]:
batchsize_2023 = len(matchid_2023) // 6
#print(batchsize_2023)
hitter_2023, pitcher_2023 = MatchPlayerRecords(matchid_2023, batchsize_2023)

Element not found for matchid 20230521OBKT02023
Element not found for matchid 20230620HTHH02023
Element not found for matchid 20230730SSWO02023
Element not found for matchid 20230902HTSK02023
Element not found for matchid 20230523LGSK02023
Completed batch 1 of 6
Completed batch 2 of 6
Completed batch 3 of 6
Completed batch 4 of 6
Completed batch 5 of 6
Completed batch 6 of 6


#### 에러 수정파트

In [233]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
h2 =  pd.DataFrame()
p2 = pd.DataFrame()
for match in ['20230521OBKT02023', '20230620HTHH02023', '20230730SSWO02023', '20230902HTSK02023', '20230523LGSK02023']:
    h, p = extractPlayerRecords(driver, match)
    h2 = pd.concat([h2, h], ignore_index=True)
    p2 = pd.concat([p2, p], ignore_index=True)

driver.quit()

In [234]:
pitcher_2023 = pd.concat([pitcher_2023, p2], ignore_index=True)
hitter_2023 = pd.concat([hitter_2023, h2], ignore_index=True)

In [235]:
pitcher_2023

,투수명,이닝,피안타,실점,자책,4사구,삼진,피홈런,타자,타수,...,패전,세이브,평균자책,연도,월,일,소속팀,상대팀,경기장,matchid
0,이정용,3,3,0,0,1,1,0,14,13,...,0.0,3.0,4.85,2023,07,02,LG,KIA,홈,20230702HTLG02023
1,정우영,1,0,0,0,0,0,0,3,3,...,4.0,0.0,4.24,2023,07,02,LG,KIA,홈,20230702HTLG02023
2,이우찬,2,2,0,0,0,0,0,6,6,...,1.0,0.0,2.89,2023,07,02,LG,KIA,홈,20230702HTLG02023
3,최동환,0 ⅓,2,1,1,0,1,0,3,3,...,0.0,0.0,2.75,2023,07,02,LG,KIA,홈,20230702HTLG02023
4,함덕주,1 ⅔,2,0,0,0,2,0,7,7,...,0.0,3.0,1.15,2023,07,02,LG,KIA,홈,20230702HTLG02023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6829,백승건,1,1,0,0,0,0,0,4,4,...,1.0,0.0,3.15,2023,05,23,SSG,LG,홈,20230523LGSK02023
6830,임찬규,6,3,1,1,1,3,1,22,21,...,0.0,0.0,2.33,2023,05,23,SSG,LG,원정,20230523LGSK02023
6831,유영찬,1,0,0,0,1,0,0,4,3,...,1.0,0.0,3.74,2023,05,23,SSG,LG,원정,20230523LGSK02023
6832,진해수,1,1,0,0,0,2,0,4,4,...,0.0,0.0,3.18,2023,05,23,SSG,LG,원정,20230523LGSK02023


In [236]:
hitter_2023

,타자명,타수,득점,안타,타점,홈런,볼넷,삼진,타율,연도,월,일,소속팀,상대팀,경기장,matchid
0,홍창기,3,1,2,0,0,0,0,0.325,2023,07,02,LG,KIA,홈,20230702HTLG02023
1,문성주,2,1,0,0,0,1,0,0.311,2023,07,02,LG,KIA,홈,20230702HTLG02023
2,김현수,3,0,1,1,0,1,0,0.299,2023,07,02,LG,KIA,홈,20230702HTLG02023
3,오스틴,4,0,2,0,0,0,0,0.300,2023,07,02,LG,KIA,홈,20230702HTLG02023
4,손호영,0,0,0,0,0,0,0,0.333,2023,07,02,LG,KIA,홈,20230702HTLG02023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18205,김기연,1,0,0,0,0,0,0,0.158,2023,05,23,SSG,LG,원정,20230523LGSK02023
18206,이재원,3,1,0,1,0,1,2,0.250,2023,05,23,SSG,LG,원정,20230523LGSK02023
18207,정주현,1,0,0,0,0,0,0,0.174,2023,05,23,SSG,LG,원정,20230523LGSK02023
18208,김민성,3,1,1,4,1,0,0,0.261,2023,05,23,SSG,LG,원정,20230523LGSK02023


In [237]:
print(len(pitcher_2023['matchid'].unique()), len(hitter_2023['matchid'].unique()))

720 720


In [238]:
hitter_2023.to_csv('hitter_2023.csv')
pitcher_2023.to_csv('pitcher_2023.csv')

#### 2018년 최종 데이터

In [239]:
month2018 = ['2018-03-13', '2018-04-01', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', '2018-09-04', '2018-10-02']
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
match_2018 = year_matchdata(driver, month2018, 10, 16)

# 정규시즌 3월 24일부터라 3월 넣고 시범경기 날짜 빼기

match_2018 = match_2018.loc[(match_2018['월'].astype(int) > 3) | ((match_2018['월'].astype(int) == 3) & (match_2018['일'].astype(int) >= 24))]
match_2018

,월,일,원정,홈,원정점수,홈점수
30,3,24,한화,키움,3,6
31,3,24,KT,KIA,5,4
32,3,24,LG,NC,2,4
33,3,24,롯데,SSG,5,6
34,3,24,삼성,두산,6,3
...,...,...,...,...,...,...
746,10,13,LG,SSG,3,2
747,10,13,롯데,KIA,5,1
748,10,13,NC,한화,8,10
749,10,13,키움,삼성,5,12


In [240]:
## 경기수 잘 뽑았는지 확인

validteam = ['KIA', 'SSG', '키움', '한화', 'LG', 'KT', 'NC', '삼성', '롯데', '두산']
for team in validteam:
    print(team, ":",len(match_2018.loc[match_2018['원정'] == team]), len(match_2018.loc[match_2018['홈'] == team]))
    

KIA : 72 72
SSG : 72 72
키움 : 72 72
한화 : 72 72
LG : 72 72
KT : 72 72
NC : 72 72
삼성 : 72 72
롯데 : 72 72
두산 : 72 72


In [257]:
match_2018.to_csv('match_2018.csv')

In [246]:
matchid_2018, dup_2018_1, dup_2018_2 = match_to_id(2018, match_2018)

중복 값 1: ['20181006HTSK02018', '20181010KTLT02018']
중복 값 2: []


In [247]:
dup1 = [s[:-5] + '1' + s[-4:] for s in dup_2018_1]
dup2 = [s[:-5] + '2' + s[-4:] for s in dup_2018_1]

error_mapping = dict(zip(dup_2018_1, dup1))
matchid_2018 = sorted([error_mapping.get(item, item) for item in matchid_2018])
matchid_2018 = matchid_2018 + dup2
len(matchid_2018)

720

In [248]:
batchsize_2018 = len(matchid_2018) // 6
hitter_2018, pitcher_2018 = MatchPlayerRecords(matchid_2018, batchsize_2018)

Element not found for matchid 20180621SKSS02018
Completed batch 1 of 6
Completed batch 2 of 6
Completed batch 3 of 6
Completed batch 4 of 6
Completed batch 5 of 6
Completed batch 6 of 6


In [249]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
h2 =  pd.DataFrame()
p2 = pd.DataFrame()
for match in ['20180621SKSS02018']:
    h, p = extractPlayerRecords(driver, match)
    h2 = pd.concat([h2, h], ignore_index=True)
    p2 = pd.concat([p2, p], ignore_index=True)

driver.quit()

In [250]:
pitcher_2018 = pd.concat([pitcher_2018, p2], ignore_index=True)
hitter_2018 = pd.concat([hitter_2018, h2], ignore_index=True)

In [255]:
print(len(pitcher_2018['matchid'].unique()), len(pitcher_2018['matchid'].unique()))

720 720


In [252]:
pitcher_2018

,투수명,이닝,피안타,실점,자책,4사구,삼진,피홈런,타자,타수,...,패전,세이브,평균자책,연도,월,일,소속팀,상대팀,경기장,matchid
0,레일리,3,8,6,6,3,4,1,21,17,...,3.0,0.0,5.53,2018,04,22,롯데,SSG,홈,20180422SKLT02018
1,배장호,3,3,2,2,2,1,1,12,10,...,1.0,0.0,7.88,2018,04,22,롯데,SSG,홈,20180422SKLT02018
2,고효준,0 ⅔,0,2,2,2,1,0,4,2,...,0.0,0.0,6.75,2018,04,22,롯데,SSG,홈,20180422SKLT02018
3,구승민,2 ⅓,2,0,0,2,1,0,10,8,...,0.0,0.0,3.86,2018,04,22,롯데,SSG,홈,20180422SKLT02018
4,박종훈,5,6,3,3,1,4,0,21,20,...,1.0,0.0,5.54,2018,04,22,롯데,SSG,원정,20180422SKLT02018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6253,박종훈,5,3,1,1,5,4,0,23,18,...,4.0,0.0,4.91,2018,06,21,삼성,SSG,원정,20180621SKSS02018
6254,정영일,1,1,0,0,1,2,0,5,4,...,0.0,0.0,4.76,2018,06,21,삼성,SSG,원정,20180621SKSS02018
6255,박정배,1,1,0,0,0,2,0,4,4,...,2.0,9.0,4.97,2018,06,21,삼성,SSG,원정,20180621SKSS02018
6256,박희수,1,0,0,0,0,1,0,3,3,...,0.0,0.0,1.26,2018,06,21,삼성,SSG,원정,20180621SKSS02018


In [253]:
hitter_2018

,타자명,타수,득점,안타,타점,홈런,볼넷,삼진,타율,연도,월,일,소속팀,상대팀,경기장,matchid
0,손아섭,4,1,1,0,0,0,1,0.312,2018,04,22,롯데,SSG,홈,20180422SKLT02018
1,전준우,1,0,0,0,0,0,0,0.226,2018,04,22,롯데,SSG,홈,20180422SKLT02018
2,문규현,4,1,2,1,0,0,0,0.225,2018,04,22,롯데,SSG,홈,20180422SKLT02018
3,채태인,4,1,1,1,0,0,2,0.286,2018,04,22,롯데,SSG,홈,20180422SKLT02018
4,이대호,4,0,2,0,0,0,1,0.388,2018,04,22,롯데,SSG,홈,20180422SKLT02018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18219,이성우,0,0,0,0,0,0,0,0.239,2018,06,21,삼성,SSG,원정,20180621SKSS02018
18220,김성현,4,0,0,1,0,0,0,0.305,2018,06,21,삼성,SSG,원정,20180621SKSS02018
18221,나주환,4,0,0,0,0,0,1,0.273,2018,06,21,삼성,SSG,원정,20180621SKSS02018
18222,최항,2,0,1,0,0,1,1,0.270,2018,06,21,삼성,SSG,원정,20180621SKSS02018


In [256]:
hitter_2018.to_csv('hitter_2018.csv')
pitcher_2018.to_csv('pitcher_2018.csv')

#### 2019년 최종 데이터

In [258]:
month2019 = ['2019-03-12', '2019-04-02', '2019-05-01', '2019-06-01', '2019-07-02', '2019-08-01', '2019-09-01', '2019-10-01']
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
match_2019 = year_matchdata(driver, month2019, 10, 3)

# 정규시즌 3월 23일부터라 3월 넣고 시범경기 날짜 빼기

match_2019 = match_2019.loc[(match_2019['월'].astype(int) > 3) | ((match_2019['월'].astype(int) == 3) & (match_2019['일'].astype(int) >= 23))]
match_2019

,월,일,원정,홈,원정점수,홈점수
37,3,23,한화,두산,4,5
38,3,23,KT,SK,4,7
39,3,23,LG,KIA,2,0
40,3,23,삼성,NC,0,7
41,3,23,키움,롯데,7,4
...,...,...,...,...,...,...
753,9,29,키움,롯데,4,1
754,9,30,롯데,LG,0,2
755,9,30,SK,한화,6,2
756,10,1,NC,두산,5,6


In [260]:
## 경기수 잘 뽑았는지 확인

validteam = ['KIA', 'SK', '키움', '한화', 'LG', 'KT', 'NC', '삼성', '롯데', '두산']
for team in validteam:
    print(team, ":",len(match_2019.loc[match_2019['원정'] == team]), len(match_2019.loc[match_2019['홈'] == team]))
    

KIA : 72 72
SK : 72 72
키움 : 72 72
한화 : 72 72
LG : 72 72
KT : 72 72
NC : 72 72
삼성 : 72 72
롯데 : 72 72
두산 : 72 72


In [261]:
match_2019.to_csv('match_2019.csv')

In [262]:
matchid_2019, dup_2019_1, dup_2019_2 = match_to_id(2019, match_2019)

중복 값 1: ['20190919OBSK02019']
중복 값 2: []


In [263]:

dup1 = [s[:-5] + '1' + s[-4:] for s in dup_2019_1]
dup2 = [s[:-5] + '2' + s[-4:] for s in dup_2019_1]

error_mapping = dict(zip(dup_2019_1, dup1))
matchid_2019 = sorted([error_mapping.get(item, item) for item in matchid_2019])
matchid_2019 = matchid_2019 + dup2
len(matchid_2019)

720

In [264]:
batchsize_2019 = len(matchid_2019) // 6
hitter_2019, pitcher_2019 = MatchPlayerRecords(matchid_2019, batchsize_2019)

Element not found for matchid 20190703LTSK02019
Element not found for matchid 20190602WOHT02019
Element not found for matchid 20190912HTOB02019
Element not found for matchid 20190404WONC02019
Element not found for matchid 20190609HTNC02019
Element not found for matchid 20190712OBLT02019
Element not found for matchid 20190712SSLG02019
Element not found for matchid 20190612OBHH02019
Element not found for matchid 20190923HHLG02019
Element not found for matchid 20190514SKNC02019
Element not found for matchid 20190712WOSK02019
Element not found for matchid 20190612SKKT02019
Element not found for matchid 20190923HTKT02019
Element not found for matchid 20190414KTSS02019
Element not found for matchid 20190820LTSK02019
Element not found for matchid 20190514SSOB02019
Element not found for matchid 20190713HHHT02019
Element not found for matchid 20190930LTLG02019
Completed batch 1 of 6
Completed batch 2 of 6
Completed batch 3 of 6
Completed batch 4 of 6
Completed batch 5 of 6
Completed batch 6 of 

In [265]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
h2 =  pd.DataFrame()
p2 = pd.DataFrame()
for match in ['20190703LTSK02019','20190602WOHT02019','20190912HTOB02019','20190404WONC02019', '20190609HTNC02019',
              '20190712OBLT02019', '20190712SSLG02019', '20190612OBHH02019', '20190923HHLG02019', '20190514SKNC02019',
              '20190712WOSK02019', '20190612SKKT02019', '20190923HTKT02019', '20190414KTSS02019', '20190820LTSK02019',
              '20190514SSOB02019', '20190713HHHT02019', '20190930LTLG02019']:
    h, p = extractPlayerRecords(driver, match)
    h2 = pd.concat([h2, h], ignore_index=True)
    p2 = pd.concat([p2, p], ignore_index=True)

driver.quit()

In [266]:
pitcher_2019 = pd.concat([pitcher_2019, p2], ignore_index=True)
hitter_2019 = pd.concat([hitter_2019, h2], ignore_index=True)

In [267]:
print(len(pitcher_2019['matchid'].unique()), len(pitcher_2019['matchid'].unique()))

720 720


In [268]:
pitcher_2019

,투수명,이닝,피안타,실점,자책,4사구,삼진,피홈런,타자,타수,...,패전,세이브,평균자책,연도,월,일,소속팀,상대팀,경기장,matchid
0,린드블럼,5 ⅔,9,2,2,3,5,0,27,24,...,0.0,0.0,3.18,2019,03,23,두산,한화,홈,20190323HHOB02019
1,김승회,0 ⅔,0,0,0,1,0,0,3,2,...,0.0,0.0,0.00,2019,03,23,두산,한화,홈,20190323HHOB02019
2,윤명준,0 ⅔,1,1,1,0,0,0,2,2,...,0.0,0.0,13.50,2019,03,23,두산,한화,홈,20190323HHOB02019
3,이현승,0 ⅔,0,0,0,0,0,0,2,1,...,0.0,0.0,0.00,2019,03,23,두산,한화,홈,20190323HHOB02019
4,박치국,0 ⅔,1,1,1,1,2,0,4,3,...,0.0,0.0,13.50,2019,03,23,두산,한화,홈,20190323HHOB02019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6198,김대현,1,0,0,0,0,0,0,3,3,...,4.0,0.0,3.78,2019,09,30,LG,롯데,홈,20190930LTLG02019
6199,고우석,1,2,0,0,0,0,0,5,5,...,2.0,35.0,1.52,2019,09,30,LG,롯데,홈,20190930LTLG02019
6200,박세웅,6,4,1,1,1,4,0,22,20,...,6.0,0.0,4.20,2019,09,30,LG,롯데,원정,20190930LTLG02019
6201,김원중,1,1,0,0,0,0,0,3,3,...,10.0,0.0,5.63,2019,09,30,LG,롯데,원정,20190930LTLG02019


In [269]:
hitter_2019

,타자명,타수,득점,안타,타점,홈런,볼넷,삼진,타율,연도,월,일,소속팀,상대팀,경기장,matchid
0,허경민,4,0,0,0,0,0,0,0.000,2019,03,23,두산,한화,홈,20190323HHOB02019
1,정수빈,3,1,1,0,0,1,0,0.333,2019,03,23,두산,한화,홈,20190323HHOB02019
2,박건우,4,2,2,2,1,0,0,0.500,2019,03,23,두산,한화,홈,20190323HHOB02019
3,김재환,2,1,0,0,0,2,1,0.000,2019,03,23,두산,한화,홈,20190323HHOB02019
4,오재일,2,1,0,0,0,2,0,0.000,2019,03,23,두산,한화,홈,20190323HHOB02019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,조홍석,1,0,0,0,0,0,0,0.117,2019,09,30,LG,롯데,원정,20190930LTLG02019
17578,신본기,2,0,0,0,0,1,1,0.257,2019,09,30,LG,롯데,원정,20190930LTLG02019
17579,김동한,2,0,0,0,0,1,0,0.284,2019,09,30,LG,롯데,원정,20190930LTLG02019
17580,나종덕,3,0,0,0,0,0,0,0.125,2019,09,30,LG,롯데,원정,20190930LTLG02019


In [270]:
hitter_2019.to_csv('hitter_2019.csv')
pitcher_2019.to_csv('pitcher_2019.csv')

#### 2020년 최종 데이터

In [41]:
month2020 = ['2020-05-01', '2020-06-02', '2020-07-01', '2020-08-01', '2020-09-01', '2020-10-01']
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
match_2020 = year_matchdata(driver, month2020, 11, 1)

# 정규시즌 5월 5일부터

match_2020 = match_2020.loc[(match_2020['월'].astype(int) > 5) | ((match_2020['월'].astype(int) == 5) & (match_2020['일'].astype(int) >= 5))]
match_2020

,월,일,원정,홈,원정점수,홈점수
5,5,5,한화,SK,3,0
6,5,5,롯데,KT,7,2
7,5,5,NC,삼성,4,0
8,5,5,두산,LG,2,8
9,5,5,키움,KIA,11,2
...,...,...,...,...,...,...
720,10,30,KT,한화,3,4
721,10,30,LG,SSG,2,3
722,10,30,NC,삼성,4,4
723,10,30,키움,두산,0,2


In [42]:
match_2020.reset_index(drop=True, inplace=True)
match_2020.loc[match_2020['원정']=='SK', '원정'] = 'SSG'
match_2020.loc[match_2020['홈']=='SK', '홈'] = 'SSG'

In [43]:
## 경기수 잘 뽑았는지 확인

validteam = ['KIA', 'SSG', '키움', '한화', 'LG', 'KT', 'NC', '삼성', '롯데', '두산']
for team in validteam:
    print(team, ":",len(match_2020.loc[match_2020['원정'] == team]), len(match_2020.loc[match_2020['홈'] == team]))
    

KIA : 72 72
SSG : 72 72
키움 : 72 72
한화 : 72 72
LG : 72 72
KT : 72 72
NC : 72 72
삼성 : 72 72
롯데 : 72 72
두산 : 72 72


In [60]:
match_2020.to_csv('match_2020.csv')

In [45]:
matchid_2020, dup_2020_1, dup_2020_2 = match_to_id(2020, match_2020)

중복 값 1: ['20200516WOLG02020', '20200611SKLG02020', '20200613KTSS02020', '20200614OBHH02020', '20200625NCKT02020', '20200625OBSK02020', '20200625WOLG02020', '20200830KTHT02020', '20200830OBLG02020', '20200904HTLT02020', '20200904SKKT02020', '20200909HHSS02020', '20200920NCLT02020', '20200927WOOB02020', '20201003LGKT02020', '20201003SSNC02020', '20201007HHHT02020', '20201010NCLG02020', '20201017SSHH02020']
중복 값 2: []


In [47]:
dup1 = [s[:-5] + '1' + s[-4:] for s in dup_2020_1]
dup2 = [s[:-5] + '2' + s[-4:] for s in dup_2020_1]

error_mapping = dict(zip(dup_2020_1, dup1))
matchid_2020 = sorted([error_mapping.get(item, item) for item in matchid_2020])
matchid_2020 = matchid_2020 + dup2
len(matchid_2020)

720

In [48]:
batchsize_2020 = len(matchid_2020) // 6
hitter_2020, pitcher_2020 = MatchPlayerRecords(matchid_2020, batchsize_2020)

Element not found for matchid 20200702SKSS02020
Element not found for matchid 20200906NCSS02020
Element not found for matchid 20200614OBHH12020
Element not found for matchid 20200830OBLG12020
Element not found for matchid 20200614OBHH22020
Element not found for matchid 20200830OBLG22020
Completed batch 1 of 6
Completed batch 2 of 6
Completed batch 3 of 6
Completed batch 4 of 6
Completed batch 5 of 6
Completed batch 6 of 6


In [55]:
check = ['20200906NCSS12020', '20200613OBHH02020', '20200829OBLG02020', '20200614OBHH02020', '20200830OBLG02020']

set(check).difference(set(hitter_2020['matchid'].unique()))

{'20200613OBHH02020',
 '20200614OBHH02020',
 '20200829OBLG02020',
 '20200830OBLG02020',
 '20200906NCSS12020'}

In [56]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
h2 =  pd.DataFrame()
p2 = pd.DataFrame()
for match in ['20200702SKSS02020', '20200906NCSS12020', '20200613OBHH02020', '20200829OBLG02020', '20200614OBHH02020', '20200830OBLG02020']:
    h, p = extractPlayerRecords(driver, match)
    h2 = pd.concat([h2, h], ignore_index=True)
    p2 = pd.concat([p2, p], ignore_index=True)

driver.quit()

In [57]:
pitcher_2020 = pd.concat([pitcher_2020, p2], ignore_index=True)
hitter_2020 = pd.concat([hitter_2020, h2], ignore_index=True)

In [58]:
print(len(hitter_2020['matchid'].unique()), len(hitter_2020['matchid'].unique()))

720 720


In [59]:
hitter_2020.to_csv('hitter_2020.csv')
pitcher_2020.to_csv('pitcher_2020.csv')

#### 2017년 최종 데이터

In [13]:
month2017 = ['2017-03-14', '2017-04-01', '2017-05-02', '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01']
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
match_2017 = year_matchdata(driver, month2017, 10, 5)


match_2017 = match_2017.loc[(match_2017['월'].astype(int) > 3) | ((match_2017['월'].astype(int) == 3) & (match_2017['일'].astype(int) >= 31))]
match_2017

,월,일,원정,홈,원정점수,홈점수
58,3,31,한화,두산,0,3
59,3,31,KIA,삼성,7,2
60,3,31,KT,SSG,3,2
61,3,31,LG,키움,2,1
62,3,31,롯데,NC,5,6
...,...,...,...,...,...,...
773,10,3,KIA,KT,10,2
774,10,3,SSG,두산,3,2
775,10,3,LG,롯데,2,4
776,10,3,NC,한화,8,8


In [16]:
match_2017.reset_index(drop=True, inplace=True)
match_2017.loc[match_2017['원정']=='SK', '원정'] = 'SSG'
match_2017.loc[match_2017['홈']=='SK', '홈'] = 'SSG'

In [23]:
match_2017.loc[ ((match_2017['홈'] == '한화') & (match_2017['원정'] == '삼성')) | ((match_2017['원정'] == '한화') & (match_2017['홈'] == '삼성'))]

,월,일,원정,홈,원정점수,홈점수
40,4,11,한화,삼성,11,8
45,4,12,한화,삼성,5,3
50,4,13,한화,삼성,1,5
203,5,19,삼성,한화,6,2
208,5,20,삼성,한화,9,8
213,5,21,삼성,한화,8,7
290,6,9,삼성,한화,8,5
295,6,10,삼성,한화,2,10
300,6,11,삼성,한화,7,4
346,6,23,한화,삼성,3,5


In [29]:
new_row = pd.DataFrame({'월': ['9'], '일': ['23'], '원정' : ['삼성'], '홈' : ['한화'], '원정점수' : ['7'], '홈점수' : ['8']})
match_2017 = pd.concat([match_2017, new_row], ignore_index=True)
len(match_2017)

721

In [31]:
match_2017['원정'].unique()

array(['한화', 'KIA', 'KT', 'LG', '롯데', 'NC', 'SSG', '삼성', '키움', '두산', '드림'],
      dtype=object)

In [30]:
## 경기수 잘 뽑았는지 확인

validteam = ['KIA', 'SSG', '키움', '한화', 'LG', 'KT', 'NC', '삼성', '롯데', '두산']
for team in validteam:
    print(team, ":",len(match_2017.loc[match_2017['원정'] == team]), len(match_2017.loc[match_2017['홈'] == team]))
    

KIA : 72 72
SSG : 72 72
키움 : 72 72
한화 : 72 72
LG : 72 72
KT : 72 72
NC : 72 72
삼성 : 72 72
롯데 : 72 72
두산 : 72 72


In [32]:
match_2017.to_csv('match_2017.csv')

In [34]:
matchid_2017, d1, d2 = match_to_id(2017, match_2017)

dup1 = [s[:-5] + '1' + s[-4:] for s in d1]
dup2 = [s[:-5] + '2' + s[-4:] for s in d2]

error_mapping = dict(zip(d1, dup1))
matchid_2017 = sorted([error_mapping.get(item, item) for item in matchid_2017])
matchid_2017 = matchid_2017 + dup2
len(matchid_2017)

중복 값 1: []
중복 값 2: []


720

In [36]:
batchsize_2017 = len(matchid_2017) // 6
hitter_2017, pitcher_2017 = MatchPlayerRecords(matchid_2017, batchsize_2017)

Element not found for matchid 20170528KTOB02017
Completed batch 1 of 6
Completed batch 2 of 6
Completed batch 3 of 6
Completed batch 4 of 6
Completed batch 5 of 6
Completed batch 6 of 6


In [37]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
h2 =  pd.DataFrame()
p2 = pd.DataFrame()
for match in ['20170528KTOB02017']:
    h, p = extractPlayerRecords(driver, match)
    h2 = pd.concat([h2, h], ignore_index=True)
    p2 = pd.concat([p2, p], ignore_index=True)

driver.quit()

In [38]:
pitcher_2017 = pd.concat([pitcher_2017, p2], ignore_index=True)
hitter_2017 = pd.concat([hitter_2017, h2], ignore_index=True)

In [39]:
print(len(pitcher_2017['matchid'].unique()), len(pitcher_2017['matchid'].unique()))

720 720


In [40]:
hitter_2017.to_csv('hitter_2017.csv')
pitcher_2017.to_csv('pitcher_2017.csv')